In [1]:
import os
import pandas as pd
import numpy as np
import re
import json
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn import preprocessing

pd.set_option('display.max_columns', None)

In [2]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords


In [3]:
PATH = sys.path[3]
PATH

'/Users/fneut/Documents/git/n26_api'

In [4]:
WORDS = stopwords.words('english') #typical words, as 'me', 'we', 'our', 'be', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should'


In [5]:

def label_transaction(transactions: dict):
    for counter, transaction_operation in enumerate(transactions):
        if "amount" in transaction_operation.keys():
            if transaction_operation["amount"] > 0:
                transactions[counter]["expense_type"] = "deposit"
            else:
                transactions[counter]["expense_type"] = "expense"
    return transactions

def remove_wording_in_transaction_label(transactions: dict):
    for counter, transaction_operation in enumerate(transactions):
        if "category" in transaction_operation.keys():
            transactions[counter]["category_preprocessed"] = " ".join(transaction_operation["category"].split("micro-v2-")[-1].split("-"))
    return transactions


def remove_commonly_used_words_from_transactions(transactions: dict):
    """
    #keeps the descriptions of the banking transaction, and removes garbage data from the transaction string 
    # like colon, words that don't give any value as a feature. 
    # [^a-zA-Z_] match everything that is NOT an uppercase or lowercase letter at the start of the word
    # i.e. "-netto" will be replaced by " netto"  
    """
    for counter, transaction_operation in enumerate(transactions):
        text_merged = ""
        for col_name in ["merchantName", "referenceText"]: 
            if col_name in transaction_operation.keys():
                text_preprocessed = " ".join([i for i in re.sub("[^a-zA-Z]"," ",transaction_operation[col_name]).split() if (i.lower() not in WORDS)]).lower()
                transactions[counter][f"{col_name}_preprocessed"] = text_preprocessed
                text_merged += f"{text_preprocessed} "
        transactions[counter]["transaction_description_merged"] = text_merged.strip()
    return transactions

In [6]:
with open(f"{PATH}/data_to_work_transactions.json") as f:
    data = json.load(f)

In [7]:
remove_commonly_used_words_from_transactions(data)
remove_wording_in_transaction_label(data)
label_transaction(data)

[{'id': '9be63438-e4fb-31b7-9384-65915fb23c30',
  'userId': 'd5b94e64-1311-47a1-989f-a90cd8b508c3',
  'type': 'PT',
  'amount': -9.45,
  'currencyCode': 'EUR',
  'originalAmount': -9.45,
  'originalCurrency': 'EUR',
  'exchangeRate': 1.0,
  'merchantCity': 'Aachen',
  'visibleTS': 1665225611331,
  'mcc': 5411,
  'mccGroup': 7,
  'merchantName': 'Netto Marken-Discount',
  'accountId': '5d4d096d-da31-4e23-b5f8-01438a55c370',
  'category': 'micro-v2-food-groceries',
  'cardId': 'f29654c5-fe14-4265-9f14-d45923518bf0',
  'referenceText': '-',
  'userCertified': 1665232571040,
  'pending': False,
  'transactionNature': 'NORMAL',
  'createdTS': 1665225611331,
  'merchantCountry': 0,
  'txnCondition': 'CARD_PRESENT',
  'smartLinkId': '9be63438-e4fb-31b7-9384-65915fb23c30',
  'linkId': '9be63438-e4fb-31b7-9384-65915fb23c30',
  'confirmed': 1665232571040,
  'merchantName_preprocessed': 'netto marken discount',
  'referenceText_preprocessed': '',
  'transaction_description_merged': 'netto marken 

In [8]:
df = pd.DataFrame.from_dict(data)

In [9]:
df.columns

Index(['id', 'userId', 'type', 'amount', 'currencyCode', 'originalAmount',
       'originalCurrency', 'exchangeRate', 'merchantCity', 'visibleTS', 'mcc',
       'mccGroup', 'merchantName', 'accountId', 'category', 'cardId',
       'referenceText', 'userCertified', 'pending', 'transactionNature',
       'createdTS', 'merchantCountry', 'txnCondition', 'smartLinkId', 'linkId',
       'confirmed', 'merchantName_preprocessed', 'referenceText_preprocessed',
       'transaction_description_merged', 'category_preprocessed',
       'expense_type', 'partnerBic', 'partnerName', 'partnerIban',
       'smartContactId', 'paymentScheme', 'mandateId', 'creditorIdentifier',
       'recurring', 'partnerAccountIsSepa', 'merchantCountryCode'],
      dtype='object')

In [10]:
df[
    [
    "amount",
    "merchantCity",
    "merchantName",
    "merchantName_preprocessed",
    "category",
    "category_preprocessed",
    "expense_type",
    "referenceText",
    "referenceText_preprocessed",
    "transaction_description_merged",
    ]
]

,amount,merchantCity,merchantName,merchantName_preprocessed,category,category_preprocessed,expense_type,referenceText,referenceText_preprocessed,transaction_description_merged
0,-9.45,Aachen,Netto Marken-Discount,netto marken discount,micro-v2-food-groceries,food groceries,expense,-,,netto marken discount
1,-27.11,Aachen,"REWE Aachen, Marktstr",rewe aachen marktstr,micro-v2-food-groceries,food groceries,expense,-,,rewe aachen marktstr
2,-1200.00,NaN,NaN,NaN,micro-v2-miscellaneous,miscellaneous,expense,NaN,NaN,
3,-5.97,Aachen,Netto Marken-Discount,netto marken discount,micro-v2-food-groceries,food groceries,expense,-,,netto marken discount
4,-15.50,Aachen,TK Kurhaus Aachen,tk kurhaus aachen,micro-v2-bars-restaurants,bars restaurants,expense,-,,tk kurhaus aachen
5,-1000.00,NaN,NaN,NaN,micro-v2-savings-investments,savings investments,expense,Scalable Capital Broker 5x savings plan,scalable capital broker x savings plan,scalable capital broker x savings plan
6,-3.58,Aachen,Netto Marken-Discount,netto marken discount,micro-v2-food-groceries,food groceries,expense,-,,netto marken discount
7,-19.71,Aachen,Netto Marken-Discount,netto marken discount,micro-v2-food-groceries,food groceries,expense,-,,netto marken discount
8,-1000.00,NaN,NaN,NaN,micro-v2-miscellaneous,miscellaneous,expense,NaN,NaN,
9,-15.99,0851832022,PAYPAL *UDEMY,paypal udemy,micro-v2-education,education,expense,-,,paypal udemy


## Training

In [11]:
df_training = pd.read_csv(f"{PATH}/training_set.csv")
df_training

,description,label
0,netto,food groceries
1,rewe,food groceries
2,aldi,food groceries
3,marken,food groceries
4,tk kurhaus,sports
5,udemy,personal development
6,tier de ride,transport
7,rent,household utilities
8,broker,investment
9,scalable capital,investment


In [12]:
vectorizer = CountVectorizer() #create the object
vectorizer.fit(df_training['description']) #Gives to each word a number. Each word is a feature, givin 113 features
X_counts_train = vectorizer.fit_transform(df_training['description']).toarray() #for each transaction, it assigns a +1 to each word in the position it was assigned. En cada transacción/fila, le asigna un "1" a una palabra en determinada posición, de las 113 palabras (features) que existen.

######################################### ASIGN LABELS TO CATEGORIES OF THE TRAINING & TESTING SET #########################################

le = preprocessing.LabelEncoder() #Encode target labels with value between 0 and n_classes-1
Y_train = le.fit_transform(df_training['label']) #assings a label to each category.

# Classifier

In [13]:
X_train = X_counts_train
gnb = GaussianNB() #Naive Bayes classifier
gnb.fit(X_train,Y_train) # Fit Gaussian Naive Bayes according to X_train and the labels

GaussianNB()

# Testing

In [14]:
X_counts_test = vectorizer.transform(df['transaction_description_merged']).toarray() #repeat the process of transforming each word to a number or index in an array
Y_predicted= gnb.predict(X_counts_test) 


# Results of the model

In [15]:
predictions = [] 
predictions_label = [] 
for counter,x in enumerate(Y_predicted):
    predictions.append(list(le.classes_)[x])
    predictions_label.append(x)

In [16]:
predictions

['food groceries',
 'food groceries',
 'investment',
 'food groceries',
 'sports',
 'investment',
 'food groceries',
 'food groceries',
 'investment',
 'personal development',
 'investment',
 'transport',
 'food groceries',
 'food groceries',
 'food groceries',
 'household utilities',
 'food groceries',
 'food groceries',
 'salary',
 'food groceries']

In [34]:
predictions_label

[0, 0, 2, 0, 5, 2, 0, 0, 2, 3, 2, 6, 0, 0, 0, 1, 0, 0, 4, 0]

In [17]:
df.columns

Index(['id', 'userId', 'type', 'amount', 'currencyCode', 'originalAmount',
       'originalCurrency', 'exchangeRate', 'merchantCity', 'visibleTS', 'mcc',
       'mccGroup', 'merchantName', 'accountId', 'category', 'cardId',
       'referenceText', 'userCertified', 'pending', 'transactionNature',
       'createdTS', 'merchantCountry', 'txnCondition', 'smartLinkId', 'linkId',
       'confirmed', 'merchantName_preprocessed', 'referenceText_preprocessed',
       'transaction_description_merged', 'category_preprocessed',
       'expense_type', 'partnerBic', 'partnerName', 'partnerIban',
       'smartContactId', 'paymentScheme', 'mandateId', 'creditorIdentifier',
       'recurring', 'partnerAccountIsSepa', 'merchantCountryCode'],
      dtype='object')

In [20]:
# predictions

In [21]:
df = df.assign(category_model = predictions, label = predictions_label)
df

,id,userId,type,amount,currencyCode,originalAmount,originalCurrency,exchangeRate,merchantCity,visibleTS,mcc,mccGroup,merchantName,accountId,category,cardId,referenceText,userCertified,pending,transactionNature,createdTS,merchantCountry,txnCondition,smartLinkId,linkId,confirmed,merchantName_preprocessed,referenceText_preprocessed,transaction_description_merged,category_preprocessed,expense_type,partnerBic,partnerName,partnerIban,smartContactId,paymentScheme,mandateId,creditorIdentifier,recurring,partnerAccountIsSepa,merchantCountryCode,category_model,label
0,9be63438-e4fb-31b7-9384-65915fb23c30,d5b94e64-1311-47a1-989f-a90cd8b508c3,PT,-9.45,EUR,-9.45,EUR,1.0,Aachen,1665225611331,5411.0,7.0,Netto Marken-Discount,5d4d096d-da31-4e23-b5f8-01438a55c370,micro-v2-food-groceries,f29654c5-fe14-4265-9f14-d45923518bf0,-,1665232571040,False,NORMAL,1665225611331,0.0,CARD_PRESENT,9be63438-e4fb-31b7-9384-65915fb23c30,9be63438-e4fb-31b7-9384-65915fb23c30,1665232571040,netto marken discount,,netto marken discount,food groceries,expense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,food groceries,0
1,4c29da2f-c15f-3cd4-a938-caa026ef7025,d5b94e64-1311-47a1-989f-a90cd8b508c3,PT,-27.11,EUR,-27.11,EUR,1.0,Aachen,1665225611331,5411.0,7.0,"REWE Aachen, Marktstr",5d4d096d-da31-4e23-b5f8-01438a55c370,micro-v2-food-groceries,f29654c5-fe14-4265-9f14-d45923518bf0,-,1665232570906,False,NORMAL,1665225611331,0.0,CARD_PRESENT,4c29da2f-c15f-3cd4-a938-caa026ef7025,4c29da2f-c15f-3cd4-a938-caa026ef7025,1665232570906,rewe aachen marktstr,,rewe aachen marktstr,food groceries,expense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,food groceries,0
2,f4787306-d9ad-4707-a65d-3dafbbf12926,d5b94e64-1311-47a1-989f-a90cd8b508c3,DT,-1200.00,EUR,NaN,NaN,NaN,NaN,1665165809396,NaN,NaN,NaN,5d4d096d-da31-4e23-b5f8-01438a55c370,micro-v2-miscellaneous,NaN,NaN,1665165809459,False,NORMAL,1665165809396,NaN,NaN,f4787306-d9ad-4707-a65d-3dafbbf12926,f4787306-d9ad-4707-a65d-3dafbbf12926,1665165809459,NaN,NaN,,miscellaneous,expense,BYLADEM1001,felipe neut dkb,DE55120300001083891984,8a252a92-c72a-437d-b888-7b2853c58d7e,SEPA,NaN,NaN,NaN,NaN,NaN,investment,2
3,92d3d50b-bad5-3815-897c-a4386fcd94d1,d5b94e64-1311-47a1-989f-a90cd8b508c3,PT,-5.97,EUR,-5.97,EUR,1.0,Aachen,1665157629291,5411.0,7.0,Netto Marken-Discount,5d4d096d-da31-4e23-b5f8-01438a55c370,micro-v2-food-groceries,f29654c5-fe14-4265-9f14-d45923518bf0,-,1665161097226,False,NORMAL,1665157629291,0.0,CARD_PRESENT,92d3d50b-bad5-3815-897c-a4386fcd94d1,92d3d50b-bad5-3815-897c-a4386fcd94d1,1665161097226,netto marken discount,,netto marken discount,food groceries,expense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,food groceries,0
4,2a043002-e5fb-38b1-8cc8-df5b61eca3cc,d5b94e64-1311-47a1-989f-a90cd8b508c3,PT,-15.50,EUR,-15.50,EUR,1.0,Aachen,1665157629291,5812.0,2.0,TK Kurhaus Aachen,5d4d096d-da31-4e23-b5f8-01438a55c370,micro-v2-bars-restaurants,f29654c5-fe14-4265-9f14-d45923518bf0,-,1665161097077,False,NORMAL,1665157629291,0.0,CARD_PRESENT,2a043002-e5fb-38b1-8cc8-df5b61eca3cc,2a043002-e5fb-38b1-8cc8-df5b61eca3cc,1665161097077,tk kurhaus aachen,,tk kurhaus aachen,bars restaurants,expense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sports,5
5,f6840f65-9a60-498b-87b4-99421531edae,d5b94e64-1311-47a1-989f-a90cd8b508c3,DD,-1000.00,EUR,NaN,NaN,NaN,NaN,1665072801801,NaN,NaN,NaN,5d4d096d-da31-4e23-b5f8-01438a55c370,micro-v2-savings-investments,NaN,Scalable Capital Broker 5x savings plan,1665072801809,False,NORMAL,1665072801801,NaN,NaN,f6840f65-9a60-498b-87b4-99421531edae,f6840f65-9a60-498b-87b4-99421531edae,1665072801809,NaN,scalable capital broker x savings plan,scalable capital broker x savings plan,savings investments,expense,BDWBDEMMXXX,Felipe Neut Montes,DE18700331002335527004,5057f059-4bae-490e-8e3a-be5a79301f84,NaN,N4T9TFRHF6JEMDMNHDJINE,DE63MUC00001787612,NaN,NaN,NaN,investment,2
6,44ba262d-6f41-3c7a-9cb2-66e39398ee15,d5b94e64-1311-47a1-989f-a90cd8b508c3,PT,-3.58,EUR,-3.58,EUR,1.0,Aachen,1665044181605,5411.0,7.0,Netto Marken-Discount,5d4d096d-da31-4e23-b5f8-01438a55

# Non relevant

In [23]:
# pd.read_csv(f"{PATH}/df.csv")

In [ ]:
for col_name in df.columns:
    print(col_name, "\n", df[col_name].value_counts(), "\n")

In [24]:
df["transaction_description_merged"].value_counts()

netto marken discount                     8
rewe aachen marktstr                      2
                                          2
tk kurhaus aachen                         1
scalable capital broker x savings plan    1
paypal udemy                              1
felipe neut montes                        1
tier de ride                              1
aldi sued                                 1
rent october felipe neut                  1
lohn gehalt                               1
Name: transaction_description_merged, dtype: int64

In [ ]:
"https://stackoverflow.com/questions/15078519/python-dictionary-passed-as-an-input-to-a-function-acts-like-a-global-in-that-fu"